In [1]:
import pandas as pd
import re
from datasets import load_from_disk

In [34]:
# Load the dataset and convert to pandas DataFrame
subset_name = "processed_Meta-Llama-3.1-8B-Instruct-evals__mbpp__details"
loaded_dataset = load_from_disk(f"data/{subset_name}")
num_examples = 250
if num_examples is not None:
    loaded_dataset = loaded_dataset.select(range(num_examples))
df_original = pd.DataFrame(loaded_dataset)
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 19 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   task_type                               250 non-null    object
 1   task_name                               250 non-null    object
 2   subtask_name                            0 non-null      object
 3   input_question                          250 non-null    object
 4   input_choice_list                       0 non-null      object
 5   input_final_prompts                     250 non-null    object
 6   input_correct_responses                 0 non-null      object
 7   output_prediction_text                  250 non-null    object
 8   output_parsed_answer                    250 non-null    object
 9   output_choice_completions               0 non-null      object
 10  output_choice_negative_log_likelihoods  0 non-null      object
 11  output

In [25]:
for i, item in df_original.iterrows():
    if i>0:
        break
    
    print(item['query'])
    print(item)

['<|start_header_id|>user<|end_header_id|>\n\nYou are an expert Python programmer, and here is your task:\nWrite a function to check if given tuple is distinct or not.\nYour code should pass the following tests:\nassert check_distinct((1, 4, 5, 6, 1, 4)) == False\nassert check_distinct((1, 4, 5, 6)) == True\nassert check_distinct((2, 3, 4, 5, 6)) == True<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n```python']
task_type                                                                        Generative
task_name                                                                         mbpp_chat
subtask_name                                                                           None
input_question                            Write a function to check if given tuple is di...
input_choice_list                                                                      None
input_final_prompts                       [<|start_header_id|>user<|end_header_id|>\n\nY...
input_correct_respons

In [89]:
# Extraction functions
def extract_context_expansion(text):
    pattern = r'<context_generation>(.*?)</context_generation>'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return f"Context expansion section not found. The original text: {text}"

def extract_test_cases(text):
    if isinstance(text, list):
        text = ' '.join(text)  # Join list elements into a single string
    else:
        text = str(text) 
    # pattern = r'assert\s+[\w_]+\(.*?\).*?(?=[\n<]|$)'
    pattern = r'assert\s+[\w_]+\s*\(.*?\)\s*==\s*.*?(?=[\n<]|$)'

    test_cases = re.findall(pattern, text)
    grouped_tests = []
    current_group = []
        
    return test_cases



In [91]:
df_cloned = df_original.copy()
# print(df_cloned.info())
# Extract test cases and add a new column to the cloned DataFrame
# df_cloned['extracted_test_cases'] = df_cloned['query'].apply(extract_test_cases)
df_cloned['extracted_test_cases'] = df_cloned['query'].apply(lambda x: extract_test_cases(x))


# Check if test cases are extracted for all rows
# assert (df_cloned['extracted_test_cases'].apply(len) > 0).sum() == 250
rows_with_test_cases_all = df_cloned['extracted_test_cases'].apply(lambda x: len(x))

print(f"Rows with extracted test cases: {rows_with_test_cases.sum()}, {rows_with_test_cases_all.sum()}")
print(f"Total rows: {len(df_cloned)}")

if rows_with_test_cases.sum() == len(df_cloned):
    print("Test cases extracted for all rows.")
else:
    print("Warning: Some rows do not have extracted test cases.")
df_cloned.info()

Rows with extracted test cases: 250, 750
Total rows: 250
Test cases extracted for all rows.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 20 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   task_type                               250 non-null    object
 1   task_name                               250 non-null    object
 2   subtask_name                            0 non-null      object
 3   input_question                          250 non-null    object
 4   input_choice_list                       0 non-null      object
 5   input_final_prompts                     250 non-null    object
 6   input_correct_responses                 0 non-null      object
 7   output_prediction_text                  250 non-null    object
 8   output_parsed_answer                    250 non-null    object
 9   output_choice_completions               0 non-null

In [93]:
output_file = 'verified_df_with_test_cases.csv'
df_cloned.to_csv(output_file, index=False)

print(f"Updated DataFrame has been saved to {output_file}")

df_cloned

Updated DataFrame has been saved to verified_df_with_test_cases.csv


,task_type,task_name,subtask_name,input_question,input_choice_list,input_final_prompts,input_correct_responses,output_prediction_text,output_parsed_answer,output_choice_completions,output_choice_negative_log_likelihoods,output_metrics,is_correct,input_question_hash,input_final_prompts_hash,benchmark_label,eval_config,context,query,extracted_test_cases
0,Generative,mbpp_chat,None,Write a function to check if given tuple is di...,None,[<|start_header_id|>user<|end_header_id|>\n\nY...,None,[\ndef check_distinct(tup):\n return len(tup)...,\ndef check_distinct(tup):\n return len(tup) ...,None,None,"{'compiles@1': 1.0, 'pass@1': 1.0}",True,5f95e606207366d0b090735637ee25085e3a99edfc4272...,[9c7412dfa641dda248586db1638287547113760c3daa7...,MBPP,"{'max_gen_len': '256', 'max_prompt_len': '4500...",[<|start_header_id|>user<|end_header_id|>\n\nY...,[<|start_header_id|>user<|end_header_id|>\n\nY...,"[assert check_distinct((1, 4, 5, 6, 1, 4)) == ..."
1,Generative,mbpp_chat,None,Write a python function to find the first non-...,None,[<|start_header_id|>user<|end_header_id|>\n\nY...,None,[\ndef first_non_repeating_character(s):\n ...,\ndef first_non_repeating_character(s):\n c...,None,None,"{'compiles@1': 1.0, 'pass@1': 1.0}",True,cc38386cf8143ac55c409cb458e395d2bb9490e34923c2...,[8a2ac8861b126e3cfa6d32d3430fc32c03495fda8f55e...,MBPP,"{'max_gen_len': '256', 'max_prompt_len': '4500...",[<|start_header_id|>user<|end_header_id|>\n\nY...,[<|start_header_id|>user<|end_header_id|>\n\nY...,"[assert first_non_repeating_character(""abcabc""..."
2,Generative,mbpp_chat,None,Write a function to check whether the given st...,None,[<|start_header_id|>user<|end_header_id|>\n\nY...,None,[\nimport re\ndef check_char(s):\n if re.ma...,\nimport re\ndef check_char(s):\n if re.mat...,None,None,"{'compiles@1': 1.0, 'pass@1': 0.0}",False,06bf66e2598332c7a63216841e42c9968ac9818103f56e...,[0f181e2f381d724e91db0ef238c09db30576ab4c46e3b...,MBPP,"{'max_gen_len': '256', 'max_prompt_len': '4500...",[<|start_header_id|>user<|end_header_id|>\n\nY...,[<|start_header_id|>user<|end_header_id|>\n\nY...,"[assert check_char(""abba"") == ""Valid"", assert ..."
3,Generative,mbpp_chat,None,Write a function to perform index wise additio...,None,[<|start_header_id|>user<|end_header_id|>\n\nY...,None,"[\ndef add_nested_tuples(tup1, tup2):\n retur...","\ndef add_nested_tuples(tup1, tup2):\n return...",None,None,"{'compiles@1': 1.0, 'pass@1': 1.0}",True,e369378b6001a801cc0ba1135a8c7b8f6c295285f78cf2...,[5bd6b4527ea9c1a1bac54e730a5500ae867de1214d0b2...,MBPP,"{'max_gen_len': '256', 'max_prompt_len': '4500...",[<|start_header_id|>user<|end_header_id|>\n\nY...,[<|start_header_id|>user<|end_header_id|>\n\nY...,"[assert add_nested_tuples(((1, 3), (4, 5), (2,..."
4,Generative,mbpp_chat,None,Write a function to compute the value of ncr%p.,None,[<|start_header_id|>user<|end_header_id|>\n\nY...,None,"[\ndef ncr_modp(n, r, p):\n if n < r:\n ...","\ndef ncr_modp(n, r, p):\n if n < r:\n ...",None,None,"{'compiles@1': 1.0, 'pass@1': 0.0}",False,855e373e328b28fede8e68c6ad20f10b18ce5f8114aff0...,[7b6d1364ec507779c60b8be0b55d5e0cb18cf1d34f882...,MBPP,"{'max_gen_len': '256', 'max_prompt_len': '4500...",[<|start_header_id|>user<|end_header_id|>\n\nY...,[<|start_header_id|>user<|end_header_id|>\n\nY...,"[assert ncr_modp(10,2,13)==6, assert ncr_modp(..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Generative,mbpp_chat,None,Write a function to split a list for every nth...,None,[<|start_header_id|>user<|end_header_id|>\n\nY...,None,"[\ndef list_split(lst, n):\n return [lst[i:...","\ndef list_split(lst, n):\n return [lst[i:i...",None,None,"{'compiles@1': 1.0, 'pass@1': 0.0}",False,cd24fd4916536d0e8433572ff2789d6466c6473961a0b0...,[9fb4993965a2e534d5331009cf09a9158a2f685e16304...,MBPP,"{'max_gen_len': '256', 'max_prompt_len': '4500...",[<|start_header_id|>user<|end_header_id|>\n\nY...,[<|start_header_id|>user<|end_header_id|>\n\nY...,"[assert list_split(['a', 'b', 'c'